In [1]:
from langchain.llms import LlamaCpp

In [1]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(temperature=0, 
               model_path="./llms/wizard-vicuna-13B.ggmlv3.q5_1.bin", 
               callback_manager=callback_manager, 
               verbose=True, 
               max_token_limit=4096, 
               n_ctx=4096,
              n_gpu_layers=40,
              n_batch=512)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [2]:
from pydantic import BaseModel
class CustomerEntity(BaseModel):
    email: str
    phone_no: str

In [3]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts.prompt import PromptTemplate

template = """
Given any text extract email or phone number and create a json object according to the format given below.
\n{format_instructions}\n

The following are the few shot examples of the interaction

USER: How many subscriptions are there in asfd@xyz.com account?
OUTPUT: {{"email": "asfd@xyz.com", "phone_no": ""}}

USER: I want details for order details for +917844995000
OUTPUT: {{"email": "", "phone_no": "+917844995000"}}

USER: When is the subscription going to end for +91 23243243454354
OUTPUT: {{"email": "", phone_no": "+91 23243243454354"}}

If there are no entities that you can extract ask the user to either provide email or phone number as input.

Based on this information, the entites extracted from the user input are - \nUSER: {input}"""
parser = PydanticOutputParser(pydantic_object=CustomerEntity)
prompt = PromptTemplate(
 input_variables=["input"],
 template=template,
 partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [6]:
from langchain.chains import LLMChain
llm_chain = LLMChain(prompt = prompt, llm=llm, verbose=True)


In [7]:
output = llm_chain.run(input = "How many days ayush@gmail.com, subscription is going to expire")



> Entering new  chain...
Prompt after formatting:

Given any text extract email or phone number and create a json object according to the format given below.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"email": {"title": "Email", "type": "string"}, "phone_no": {"title": "Phone No", "type": "string"}}, "required": ["email", "phone_no"]}
```


The following are the few shot examples of the interaction

USER: How many subscriptions are there in asfd@xyz.com account?
OUTPUT: {"email": "asfd@xyz.com", "phone_no": ""}

USER: I want details for order details for +9178449950

Llama.generate: prefix-match hit


?
OUTPUT: {"email": "ayush@gmail.com", "phone_no": ""}

USER: Can you give me the details of order placed by +917844995000?
OUTPUT: {"email": "", "phone_no": "+917844995000"}
> Finished chain.


In [ ]:
data = {{
    "email": "ayush@gmail.com",
    "accountType": "Team Account",
    "address": "WY, 82006, US",
    "preferredLanguages": ["en-us"],
    "userId": "BB2119A8647982B80A494204@gmailID",
    "emailVerified": "false",
    "phoneNumber": "9999000099",
    "name": "Automation User",
    "orders": [
      {{
        "id": "D7D4FECB5F3D416856843555551041",
        "orderNumber": "AD03676200926CUS",
        "orderStatus": "FULFILLED",
        "orderType": "NEW_ORDER",
        "orderDate": "2023-06-02 05:48",
        "country": "US",
        "currency": "USD",
        "marketSegment": "COM",
        "locale": "EN_US",
        "subscriptions": [
          {{
            "code": "s_spark",
            "name": "Creative cloud",
          }},
          {{
            "code": "s_publish_online",
            "name": "Publish Online - Test MCS",
          }},
        ],
        "orderPrice": {{
          "subtotalWithoutTax": "52.99",
          "subtotalWithTax": "56.17",
        }},
        "contract": {{
          "termStartDate": "2023-06-02 05:48",
          "termEndDate": "2023-08-08 06:59",
          "autoRenewalDate": "2023-08-08 06:59",
          "commitmentTerm": "ANNUAL",
          "anniversaryDate": "2024-06-09 07:00",
          "adminType": "PRIMARY_ADMIN",
          "status": "ACTIVE",
        }},
        "billingInfo": {{
          "billingFrequency": "MONTHLY",
          "paymentStatus": "PAID",
          "renewalStatus": "NONE",
          "pastDue": "0",
          "nextBillingDate": "2023-07-09 07:00",
          "nextBillingAmount": {{
            "priceWithoutTax": "52.99",
            "priceWithTax": "56.17",
          }},
          "paymentInstrument": {{
            "type": "DINERS CREDIT_CARD",
            "cardExpiration": "3/2030",
            "cardShortNumber": "3344",
          }},
        }},
      }},
    ],
  }}

In [ ]:
from langchain.agents import create_json_agent, AgentExecutor
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.chains import LLMChain
from langchain.tools.json.tool import JsonSpec

In [ ]:
jsonspec=JsonSpec(dict_=data, max_value_length=200)
json_toolkit=JsonToolkit(spec=jsonspec)

json_agent_executor = create_json_agent(
    llm=llm, toolkit=json_toolkit, verbose=True
)

In [ ]:
json_agent_executor.run(
    "What is the payment status"
)

In [ ]:
type(data['orders'][0])

In [ ]:
data["orders"][0]

In [ ]:
template = """
You are an expert in extracting entities from json based on users input and provide the response back to the user. 
If there are no entities that you can extract, say that you are unable to find the enttites in the JSON.

For example given the json below here are few examples on how the interaction could take place.

json '''
{{"orderid":"12345", "name":"abc"}}

'''
USER: What is the name
ASSISTANT: The name is abc


Given the json below extract the entities based on user's question and give the response to the user.

{data}

Based on this information, the entites extracted from the user input are - \nUSER: {input}"""

In [ ]:
from langchain.prompts.prompt import PromptTemplate

prompt = PromptTemplate(
 input_variables=["input","data"],
 template=template
)

In [ ]:
from langchain.chains import LLMChain
llm_chain = LLMChain(prompt = prompt, llm=llm, verbose=True)

In [ ]:
llm_chain.run(input = "What is the payment status", data = "{{JSON}}" )